# Unfitted space-time finite elements
In this example we consider a moving domain problem with homogeneous Neumann boundary conditions:

$$
\left\{
\begin{aligned}
\partial_t u + \mathbf{w} \cdot \nabla u - \alpha \Delta u &= f \quad \text{ in } \Omega(t),  & \\
~ \partial_{\mathbf{n}} u &=  0  \quad \text{ on } \partial \Omega(t), & \\
u &= u_0  \quad \text{at } t=0, & \\
\end{aligned}\right.
$$

$$
\operatorname{div}(\mathbf{w}) = 0  \quad \text{ in } \Omega(t),  \quad \mathbf{w} \cdot n = \mathcal{V}_n \text{ on } \partial \Omega(t).
$$

We consider a basic P1 version of a space-time discretization as discussed in [1]. The version in [1] (also implemented in `NGSolve` and `ngsxfem`) extends this approach to higher order in space and time.

#### Literature:
[1]: J. Preuß. Higher order unfitted isoparametric space-time FEM on moving domains. Master thesis, University of Göttingen, 2018.



In [ ]:
# unfitted Heat equation with Neumann b.c.
from netgen.geom2d import SplineGeometry
from netgen.meshing import MeshingParameters
from ngsolve import *

from ngsolve.internal import *
from xfem import *
from math import pi

import netgen.gui 
%gui tk

ngsglobals.msg_level = 1

## Background geometry and mesh:

* We consider a simple square as background domain an use a simple mesh for that.
* The space-time method uses tensor-product elements. Hence, we do not need space-time meshes.

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1,-1],[1,1])
ngmesh = square.GenerateMesh(maxh=0.08, quad_dominated=False)
mesh = Mesh (ngmesh)

#alternatively: quad mesh:
#ngmesh = square.GenerateMesh(maxh=1.8, quad_dominated=True)
#mesh = Mesh (ngmesh)
#for i in range(4):
#    mesh.Refine()
h = specialcf.mesh_size
Draw(mesh)

## Handling of the time variable

For the handling of the space-time integration we use the following rules:
 * every time step is formulated with respect to the reference interval $[0,1)$ in time
 
 * Example: $t_{n-1} = 0.4$, $t=0.55$, $\Delta t = 0.2$ $\quad \longrightarrow \quad$ $\hat{t} = 0.75$.
 
 * $\hat{t}$ is the `ReferenceTimeVariable`
 
 * We define $t_{old}(=t_{n-1})$ and $\Delta t$ as a `Parameter`, s.t. we can change the time interval later

In [ ]:
#### expression for the time variable: 
coef_told = Parameter(0)
coef_delta_t = Parameter(0)
tref = ReferenceTimeVariable()
t = coef_told + coef_delta_t*tref

## Data functions (depending on $t$)

In [ ]:
r0 = 0.5

rho =  CoefficientFunction((1/(pi))*sin(2*pi*t))
#convection velocity:
d_rho = CoefficientFunction(2*cos(2*pi*t))
w = CoefficientFunction((0,d_rho)) 

# level set
r = sqrt(x**2+(y-rho)**2)
levelset= r - r0

# diffusion coefficient
alpha = 1

# solution and r.h.s.
Q = pi/r0   
u_exact = cos(Q*r) * sin(pi*t)
coeff_f = (Q/r * sin(Q*r) + (Q**2) * cos(Q*r)) * sin(pi*t) + pi * cos(Q*r) * cos(pi*t)
u_init = u_exact

### Discretization parameters (orders)

In [ ]:
# polynomial order in time
k_t = 1
# polynomial order in space
k_s = 1
# polynomial order in time for level set approximation
lset_order_time = 1

## Space-Time finite elements

* For the construction of a space-time `FESpace` we can combine any spatial `FESpace` with a scalar `FiniteElement` in a tensor-product fashion.
* Here, we use a nodal `FiniteElement` to simplify the extraction of spatial functions at fixed times.

In [ ]:
# spatial FESpace for solution
fes1 = H1(mesh, order=k_s)
# time finite element (nodal!)
tfe = ScalarTimeFE(k_t) 
# space-time finite element space
st_fes = SpaceTimeFESpace(fes1,tfe, flags = {"dgjumps": True})

## Space-time geometry description
For the level set description of the geometry we use a space-time description on every time slab.
 * `CreateTimeRestrictedGF` generates a *spatial* GridFunction corresponding to the spatial `FESpace`

In [ ]:
lset_p1 = GridFunction(st_fes)
lset_top = CreateTimeRestrictedGF(lset_p1,1.0)
lset_bottom = CreateTimeRestrictedGF(lset_p1,0.0)

The following dictionaries correspond to the following integration domains:
* $Q_n$ : `lset_neg` (depends on space-time GridFunction `lset_p1`)
* $\Omega(t_n)$ : `lset_top` (depends on spatial GridFunction `lset_top`)
* $\Omega(t_{n-1})$ : `lset_bottom` (depends on spatial GridFunction `lset_bottom`)
        

In [ ]:
lset_neg = { "levelset" : lset_p1, "domain_type" : NEG}
lset_neg_bottom = { "levelset" : lset_bottom, "domain_type" : NEG}
lset_neg_top = { "levelset" : lset_top, "domain_type" : NEG}

Draw(lset_top,mesh,"lset")
Draw(IfPos(-lset_top,CoefficientFunction((0,0)),CoefficientFunction((float('nan'),float('nan')))),mesh,"filter")
visoptions.deformation = 1
visoptions.vecfunction = "filter"

### Space-Time version of the `CutInfo` class
The `CutInfo` class also works for space-time geometries. Its initialization is trivial:

In [ ]:
ci = CutInfo(mesh,time_order=0)

To Update the slab geometry later on we do the following:

In [ ]:
def UpdateTimeSlabGeometry():
    SpaceTimeInterpolateToP1(levelset,coef_told,told,delta_t,lset_p1)
    RestrictGFInTime(spacetime_gf=lset_p1,reference_time=0.0,space_gf=lset_bottom)
    RestrictGFInTime(spacetime_gf=lset_p1,reference_time=1.0,space_gf=lset_top)

    # update markers in (space-time) mesh
    ci.Update(lset_p1,time_order=0)    

Notice:
    * `coef_told`,`told`,`delta_t` are used to work with a variable time inside a time slab

### Solution GridFunction

In [ ]:
gfu = GridFunction(st_fes)
u_last = CreateTimeRestrictedGF(gfu,0)
Draw(u_last, mesh,"u", sd=2, autoscale=False, min = -1, max = 1)

### Collection of integrator types
The arising integrals will be categorized:

In [ ]:
hasneg_integrators_a = []
hasneg_integrators_f = []
patch_integrators_a = []

## Variational formulation

Now we would like to derive a suitable variational formulation on the time slabs $Q^{n}$. 

We start by multiplying the equation  
\begin{equation*}
\partial_{t} u- \alpha \Delta{u} + w \cdot \nabla{u} = f \quad  in \quad \Omega(t),   \qquad  t \in [0,T] 
\end{equation*}
by a test function $v$ and perform integration by parts. 

Due to homogeneous Neumann boundary conditions this leads to: 
\begin{equation*}
(\partial_{t} u, v)_{Q^n} + \alpha (\nabla{u},\nabla{v})_{Q^n}   + (w \cdot \nabla{u},v)_{Q^n} = (f,v)_{Q^n}.
\end{equation*}

## Upwind DG in time
The operator $(\nabla,\partial_{t})$ acts as a convective term in the space-time domain. We can integrate the whole term by parts and obtain:
 
\begin{alignat*}{2}\begin{aligned} & (\partial_{t}u,  v)_{Q^{n}} + ( w \cdot \nabla{u}, v)_{Q^{n}} \\
 &= -(u, \partial_{t} v)_{Q^{n}} + (u_{-}^{n},v_{-}^{n})_{\Omega(t_{n})} - (u_{-}^{n-1},v_{+}^{n-1})_{\Omega(t_{n-1})} - (u, \nabla{v} \cdot w)_{Q^{n}}.           \\  \end{aligned}  \end{alignat*} 
 
 Here it was used that the velocity of the boundary $\partial \Omega(t)$ in normal direction coincides with $w \cdot n$ where $n$ is the normal of $\Omega(t)$.
 <center> ![alt](graphics/limits-time-slab.png) </center>

## Ghost penalty stabilization
To gain sufficient control on all space-time d.o.f.s we add a so-called *Ghost-Penalty* stabilization 
as in [1]. Adding the stabilization, the variational formulation on the time slabs becomes:
 
\begin{alignat*}{3}
\begin{aligned}
 &-(u, \partial_{t} v)_{Q^{n}} + \alpha (\nabla{u},\nabla{v})_{Q^{n}} - (u, \nabla{v} \cdot w)_{Q^{n}}  + (u^{n}_{-},v^{n}_{-})_{\Omega^{n}} + s_h(u,v) \\
 &= (f,v)_{Q^{n}}  +  (u^{n-1}_{-},v^{n-1}_{+})_{\Omega^{n-1}}          \\
\end{aligned}  
\end{alignat*}
$$
\text{with} \qquad\qquad
s_h(u,v) =   \sum\limits_{F \in F_{h}}{ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\omega_F}{  h^{-2} [\![ u]\!] \, [\![ v]\!]         \, d\mathbf{x} \, dt.  } }		}                 \\
$$
where $[\![u]\!]$ is the difference of $u|_{T_1}$ and $u|_{T_2}$ (interpreted as polynomials $\in \mathbb{R}^d$).
<center>![alt](graphics/macro-element.png)</center>

### Implementation of space-time integrals

In [ ]:
def SpaceTimeNegBFI(form):
    return SymbolicBFI(levelset_domain = lset_neg, form = form.Compile(), time_order=2*k_t)
u,v = st_fes.TnT()

#### Transformation from reference interval to $(t_{n-1},t_n)$:
$$
(x,\hat{t}) \to (x,t_{n-1} + \hat{t} \Delta t), \qquad v(x,t) = \hat{v}(x,\hat{t}), \quad u(x,t) = \hat{u}(x,\hat{t})
$$

First integral:
$$
-(u, \partial_{t} v)_{Q^{n}} \qquad = \qquad -(\hat{u}, \partial_{t} \hat v)_{\hat Q^{n}}
$$

In [ ]:
hasneg_integrators_a.append(SpaceTimeNegBFI(form = -u*dt(v)))

Second integral:
$$- (u, \nabla{v} \cdot w)_{Q^{n}}  \qquad = \qquad - \Delta t (\hat{u}, \nabla \hat v \cdot w)_{\hat Q^{n}} $$

In [ ]:
hasneg_integrators_a.append(SpaceTimeNegBFI(form = -coef_delta_t*u*InnerProduct(w,grad(v))))

Third integral:
$$
\alpha (\nabla{u},\nabla{v})_{Q^{n}}  \qquad = \qquad \alpha \Delta t (\nabla \hat{u}, \nabla \hat v)_{\hat Q^{n}} 
$$

In [ ]:
hasneg_integrators_a.append(SpaceTimeNegBFI(form = coef_delta_t*alpha*grad(u)*grad(v)))

Fourth integral:
$$(u^{n}_{-},v^{n}_{-})_{\Omega^{n}}, \qquad u^{n}_- = \hat u(\cdot,\hat t = 1), \qquad v^{n}_- = \hat v(\cdot,\hat t = 1)$$



In [ ]:
hasneg_integrators_a.append(SymbolicBFI(levelset_domain = lset_neg_top, form = fix_t(u,1)*fix_t(v,1)))

Fifth integral:
$$ s_h(u,v) =   \sum\limits_{F \in F_{h}}{ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\omega_F}{  h^{-2} [\![ u]\!] \, [\![ v]\!]         \, d\mathbf{x} \, dt.  }}} =   \sum\limits_{F \in F_{h}}{ \Delta t \ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\omega_F}{  h^{-2} [\![ \hat u]\!] \, [\![ \hat v]\!]         \, d\mathbf{x} \, dt.  }}}  $$

In [ ]:
patch_integrators_a.append(SymbolicFacetPatchBFI(form = coef_delta_t*0.05*h**(-2)*(u-u.Other())*(v-v.Other()),
                                                 skeleton=False, time_order=2*k_t)) 

Sixth integral:
$$ (f,v)_{Q^{n}} \qquad = \qquad \Delta t (f, \hat v)_{\hat Q^{n}} 
$$

In [ ]:
hasneg_integrators_f.append(SymbolicLFI(levelset_domain = lset_neg, form = coef_delta_t*coeff_f*v, time_order=2*k_t)) 

Seventh integral:
$$ (u^{n-1}_{-},v^{n-1}_{+})_{\Omega^{n-1}}, \qquad v^{n}_+ = \hat v(\cdot,\hat t = 0)$$

In [ ]:
hasneg_integrators_f.append(SymbolicLFI(levelset_domain = lset_neg_bottom,form = u_last*fix_t(v,0)))

### Put integrals into bilinear and linear forms

In [ ]:
a = BilinearForm(st_fes,check_unused=False,symmetric=False)
for integrator in hasneg_integrators_a + patch_integrators_a:
    a += integrator

f = LinearForm(st_fes)

for integrator in hasneg_integrators_f:
    f += integrator

### To setup the linear systems in every time step we have to
* Update the element markers (`CutInfo` does this on `Update`)
* Update the facet markers
* Re-Set the integration elements (`definedonelements`)
* Re-Assemble the system

In [ ]:
def UpdateLinearSystems():
    # re-compute the facets for stabilization:
    ba_facets = GetFacetsWithNeighborTypes(mesh,a=ci.GetElementsOfType(HASNEG),
                                                b=ci.GetElementsOfType(IF))

    # re-set definedonelements-markers according to new markings:
    for integrator in hasneg_integrators_a + hasneg_integrators_f:
        integrator.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
    for integrator in patch_integrators_a:
        integrator.SetDefinedOnElements(ba_facets)

    # assemble linear system
    a.Assemble()
    f.Assemble()


### To solve the linear systems in every time step
* need to update the dof markers (`active_dofs`)
* solve the linear system

In [ ]:
def SolveForTimeSlab():
    # re-evaluate the "active dofs" in the space time slab
    active_dofs = GetDofsOfElements(st_fes,ci.GetElementsOfType(HASNEG))
    # solve linear system
    inv = a.mat.Inverse(active_dofs,inverse="umfpack")
    gfu.vec.data =  inv * f.vec   

### At the end of every time step, we
* store the solution at $t_n$ into a (purely) spatial `GridFunction` (to be used in next time step)
* compute the error
* update visualization

In [ ]:
def FinalizeStep():
    RestrictGFInTime(spacetime_gf=gfu,reference_time=1.0,space_gf=u_last)   
    # compute error at end of time slab
    l2error = sqrt(Integrate(lset_neg_top,(u_exact-u_last)*(u_exact-u_last),mesh))
    # print time and error
    print("\rt = {0:10}, l2error = {1:20}".format(told,l2error),end="")

    Redraw(blocking=True)

### The final time loop

In [ ]:
tend = 1
delta_t = tend/32
coef_delta_t.Set(delta_t)
tnew = 0
told = 0

In [ ]:
u_last.Set(u_init) 
with TaskManager():
    while tend - told > delta_t/2:
        UpdateTimeSlabGeometry()
        UpdateLinearSystems()
        SolveForTimeSlab()       
        told = told + delta_t
        coef_told.Set(told)
        FinalizeStep()
print("")       